# Curso de Python Avanzado

<img src="https://amei.mx/wp-content/uploads/2016/08/Scotiabank-logo.jpg" alt="Scotiabank Logo" height="60">
<img src="https://www.belatrix.com/wp-content/uploads/2023/08/belatrix-logosweb-1.png" alt="Belatrix Logo" height="60">


**[Scotiabank](https://www.scotiabank.com.mx)** | **[Belatrix](https://www.belatrix.com)**

Instructor: [Alan Badillo Salas](alan@nomadacode.com)

## Bienvenida

Bienvenidos al curso de **Python Avanzado** para Scotiabank, brindado por Belatrix.

En este curso aprenderás a desarrollar una programación avanzada con Python.

El curso está dirigido a profesionales relacionados al área de desarrollo, soporte y análisis de datos.

Se requieren conocimientos previos de Python Intermedio para poder cubrir satisfactoriamente este curso.

## Temario

**Módulo 1: Programación Funcional en Python**

1. Introducción a la programación funcional
2. Funciones de orden superior
3. Lambdas y expresiones generadoras

**Módulo 2: Diseño de Patrones Avanzados**

1. Patrones de diseño comunes en Python
2. Aplicación práctica de patrones en el desarrollo de software

**Módulo 3: Manipulación Eficiente de Datos**

1. Uso avanzado de NumPy y Pandas
2. Operaciones vectorizadas y broadcasting

**Módulo 4: Concurrencia y Paralelismo en Python**

1. Hilos y procesos en Python
2. Multiprocessing y Asyncio

**Módulo 5: Optimización de Código**

1. Estrategias para mejorar el rendimiento
2. Perfilado de código y herramientas de optimización

**Módulo 6: Seguridad en Desarrollo Python**

1. Principios básicos de seguridad
2. Mejores prácticas de codificación segura

**Módulo 7: Despliegue y Escalabilidad**

1. Configuración de entornos de producción
2. Estrategias para escalabilidad horizontal y vertical

# Módulo 5: Benchmarking y elaboración de perfiles

```text
    - Diseñando tu aplicación
    - Escribir pruebas y puntos de referencia
    - Escribir mejores pruebas y puntos de referencia con pytest-benchmark
    - Encontrar cuellos de botella con cProfile
    - Optimizando nuestro código
    - Usando el módulo dis
    - Generación de perfiles de uso de memoria con memory_profiler
```

## Introducción

El "Benchmarking" es el proceso de identificar las partes lentas de un programa, lo cual es la tarea más importante para acelerar el rendimiento del código. En muchos casos, el código que causa que una aplicación se vuelva más lenta es una pequeña fracción del programa total. Al reconocer estas secciones críticas, es posible concentrarse en las partes que necesitan mejoras sin perder tiempo en optimizaciones muy específicas.

El "Profiling" es una técnica que permite identificar las partes de una aplicación que consumen más recursos. Un "profiler" es un programa que ejecuta una aplicación y sigue cuánto tiempo tarda en ejecutarse cada función, lo que permite detectar las funciones en las que la aplicación gasta la mayor parte de su tiempo.

Python ofrece diversas herramientas para encontrar estos cuellos de botella y medir métricas de rendimiento importantes. En el texto, se menciona el uso del módulo estándar "cProfile" y el paquete de terceros "line_profiler". También se aborda cómo perfilar el consumo de memoria de una aplicación mediante la herramienta "memory_profiler". Otra herramienta útil mencionada es "KCachegrind", que permite mostrar gráficamente los datos producidos por diferentes "profilers".

Por último, los "benchmarks" son pequeños scripts utilizados para evaluar el tiempo total de ejecución de una aplicación. El texto promete enseñar cómo escribir "benchmarks" y cómo usarlos para medir con precisión el tiempo de ejecución de los programas.

## Diseñando tu aplicación

### Crear un simulador de partículas

> `CLASE Particle`

```py
class Particle:
    def __init__(self, x, y, ang_vel):
        self.x = x
        self.y = y
        self.ang_vel = ang_vel
```

> `CLASE ParticleSimulator`

```py
class ParticleSimulator:
    def __init__(self, particles):
        self.particles = particles
    def evolve(self, dt):
        timestep = 0.00001
        nsteps = int(dt/timestep)
    
        for i in range(nsteps):
            for p in self.particles:
                # 1. calculate the direction
                norm = (p.x**2 + p.y**2)**0.5
                v_x = -p.y/norm
                v_y = p.x/norm
                # 2. calculate the displacement
                d_x = timestep * p.ang_vel * v_x
                d_y = timestep * p.ang_vel * v_y
                p.x += d_x
                p.y += d_y
                # 3. repeat for all the time steps
```

### Visualizar la simulación

> `FUNCIÓN visualize`

```py
from matplotlib import pyplot as plt
from matplotlib import animation

def visualize(simulator):
    X = [p.x for p in simulator.particles]
    Y = [p.y for p in simulator.particles]
    
    fig = plt.figure()
    ax = plt.subplot(111, aspect='equal')
    line, = ax.plot(X, Y, 'ro')
    
    # Axis limits
    plt.xlim(-1, 1)
    plt.ylim(-1, 1)

    # It will be run when the animation starts
    def init():
        line.set_data([], [])
        return line, # The comma is important!
    
    def animate(i):
        # We let the particle evolve for 0.01 time units
        simulator.evolve(0.01)
        X = [p.x for p in simulator.particles]
        Y = [p.y for p in simulator.particles]
        line.set_data(X, Y)
        return line,

    # Call the animate function each 10 ms
    anim = animation.FuncAnimation(fig,
        animate, init_func=init, blit=True,
        interval=10)

    plt.show()
```

> `TEST test_visualize`

```py
def test_visualize():
    particles = [
                    Particle(0.3, 0.5, 1),
                    Particle(0.0, -0.5, -1),
                    Particle(-0.1, -0.4, 3)
    ]
    simulator = ParticleSimulator(particles)
    visualize(simulator)
    
if __name__ == '__main__':
    test_visualize()
```

## Escribir `tests` y `benchmarks`

> `TEST test_evolve`

```py
def test_evolve():
    particles = [Particle( 0.3,  0.5, +1),
                    Particle( 0.0, -0.5, -1),
                    Particle(-0.1, -0.4, +3)
        ]

    simulator = ParticleSimulator(particles)
    simulator.evolve(0.1)
    p0, p1, p2 = particles
    
    def fequal(a, b, eps=1e-5):
        return abs(a - b) < eps

    assert fequal(p0.x, 0.210269)
    assert fequal(p0.y, 0.543863)
    assert fequal(p1.x, -0.099334)
    assert fequal(p1.y, -0.490034)
    assert fequal(p2.x,  0.191358)
    assert fequal(p2.y, -0.365227)

if __name__ == '__main__':
    test_evolve()
```

> `BENCHMARK benchmark`

```py
from random import uniform

def benchmark():
    particles = [
        Particle(uniform(-1.0, 1.0), uniform(-1.0, 1.0),
        uniform(-1.0, 1.0))
        for i in range(1000)]
    simulator = ParticleSimulator(particles)
    simulator.evolve(0.1)

if __name__ == '__main__':
    benchmark()
```

### Medir el tiempo del `benchmark`

> `TIME benchmark`

```bash
time python benchmarck.py
```

```text
    --- SALIDA ---

    real    0m3.544s
    user    0m3.478s
    sys     0m0.030s
```

> `INSTALL IPython`

```bash
sudo apt install ipython3
```

> `IPython timeit`

```py
[1] from benchmark import benchmark
[2] %timeit benchmark()

# %timeit
#   -n <loops>
#   -r <runs>
```

## Escribir mejores pruebas con `pytest-benchmark`

```text
    REFERENCIAS:

     http://pytest-benchmark.readthedocs.io/en/stable/usage.html
```

> `INSTALL pytest-benchmark`

```bash
pip install pytest

pip install pytest-benchmark
```

> `TEST test_evolve`

```bash
python -m pytest test_evolve.py::test_evolve
```

```text
    --- SALIDA ---

    ================= test session starts ===================================
    platform linux2 -- Python 2.7.18, pytest-4.6.9, py-1.8.1, pluggy-0.13.0
    rootdir: /mnt/d/GitHub/python-avanzado-2023/c101
    collected 1 item

    test_evolve.py .                                                   [100%]

    =============== 1 passed in 0.12 seconds ================================
```

> `TEST test_evolve_pro`

```py
def test_evolve_pro(benchmark):
    particles = [Particle( 0.3,  0.5, +1),
                    Particle( 0.0, -0.5, -1),
                    Particle(-0.1, -0.4, +3)
        ]
        
    simulator = ParticleSimulator(particles)

    benchmark(simulator.evolve, 0.1)
```

```bash
python -m pytest test_evolve.py::test_evolve_pro
```

```text
============================= test session starts ==============================
platform linux -- Python 3.8.10, pytest-7.4.0, pluggy-1.2.0
benchmark: 4.0.0 (defaults: timer=time.perf_counter disable_gc=False min_rounds=5 min_time=0.000005 max_time=1.0 calibration_precision=10 warmup=False warmup_iterations=100000)
rootdir: /mnt/d/GitHub/python-avanzado-2023/c101
plugins: benchmark-4.0.0
collected 1 item

test_evolve.py .                                                         [100%]


---------------------------------------------- benchmark: 1 tests ---------------------------------------------
Name (time in ms)         Min      Max     Mean  StdDev   Median     IQR  Outliers      OPS  Rounds  Iterations
---------------------------------------------------------------------------------------------------------------
test_evolve_pro       10.3480  14.1111  10.8215  0.4497  10.7158  0.2981       8;6  92.4082      96           1
---------------------------------------------------------------------------------------------------------------

Legend:
  Outliers: 1 Standard Deviation from Mean; 1.5 IQR (InterQuartile Range) from 1st Quartile and 3rd Quartile.
  OPS: Operations Per Second, computed as 1 / Mean
============================== 1 passed in 2.10s ===============================
```

## Encontrar los cuellos de botella con `cProfile`

> `PROFILLING benchmark`

```bash
python3 -m cProfile benchmark.py
```

```text
    --- SALIDA ---

         8070 function calls (8041 primitive calls) in 3.582 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        8    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:103(release)
        8    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:143(__init__)
        8    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:147(__enter__)
        8    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:151(__exit__)
        8    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:157(_get_module_lock)
        8    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:176(cb)
     12/3    0.000    0.000    0.001    0.000 <frozen importlib._bootstrap>:211(_call_with_frames_removed)        
       52    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:222(_verbose_message)
        4    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:232(_requires_builtin_wrapper)        
        8    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:342(__init__)
        4    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:35(_new_module)
        8    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:376(cached)
        8    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:389(parent)
        8    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:397(has_location)
        4    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:406(spec_from_loader)
        8    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:477(_init_module_attrs)
        8    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:549(module_from_spec)
        8    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:58(__init__)
      8/3    0.000    0.000    0.006    0.002 <frozen importlib._bootstrap>:650(_load_unlocked)
        8    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:725(find_spec)
        4    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:746(create_module)
        4    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:754(exec_module)
        4    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:771(is_package)
        8    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:78(acquire)
        4    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:800(find_spec)
       16    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:863(__enter__)
       16    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:867(__exit__)
        8    0.000    0.000    0.002    0.000 <frozen importlib._bootstrap>:890(_find_spec)
      8/3    0.000    0.000    0.008    0.003 <frozen importlib._bootstrap>:956(_find_and_load_unlocked)
      8/3    0.000    0.000    0.008    0.003 <frozen importlib._bootstrap>:986(_find_and_load)
        4    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap_external>:1004(__init__)
        4    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap_external>:1029(get_filename)
        4    0.001    0.000    0.004    0.001 <frozen importlib._bootstrap_external>:1034(get_data)
        4    0.000    0.000    0.001    0.000 <frozen importlib._bootstrap_external>:1075(path_stats)
       44    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap_external>:121(_path_join)
       44    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap_external>:123(<listcomp>)
        8    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap_external>:127(_path_split)
       16    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap_external>:129(<genexpr>)
       10    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap_external>:1330(_path_importer_cache)   
       16    0.000    0.000    0.003    0.000 <frozen importlib._bootstrap_external>:135(_path_stat)
        4    0.000    0.000    0.002    0.001 <frozen importlib._bootstrap_external>:1367(_get_spec)
        4    0.000    0.000    0.002    0.001 <frozen importlib._bootstrap_external>:1399(find_spec)
        4    0.000    0.000    0.001    0.000 <frozen importlib._bootstrap_external>:145(_path_is_mode_type)      
        4    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap_external>:1493(_get_spec)
        8    0.000    0.000    0.002    0.000 <frozen importlib._bootstrap_external>:1498(find_spec)
        4    0.000    0.000    0.001    0.000 <frozen importlib._bootstrap_external>:154(_path_isfile)
        4    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap_external>:175(_path_isabs)
        8    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap_external>:354(cache_from_source)       
        4    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap_external>:484(_get_cached)
        4    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap_external>:516(_check_name_wrapper)     
        4    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap_external>:553(_classify_pyc)
        4    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap_external>:586(_validate_timestamp_pyc)
        4    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap_external>:638(_compile_bytecode)       
        8    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap_external>:68(_relax_case)
        4    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap_external>:689(spec_from_file_location)
       12    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap_external>:79(_unpack_uint32)
        4    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap_external>:839(create_module)
      4/3    0.000    0.000    0.006    0.002 <frozen importlib._bootstrap_external>:842(exec_module)
        4    0.000    0.000    0.006    0.001 <frozen importlib._bootstrap_external>:914(get_code)
        1    0.000    0.000    3.582    3.582 benchmark.py:1(<module>)
        1    0.000    0.000    3.574    3.574 benchmark.py:6(benchmark)
        1    0.000    0.000    0.001    0.001 benchmark.py:7(<listcomp>)
        1    0.000    0.000    0.000    0.000 bisect.py:1(<module>)
        1    0.000    0.000    0.000    0.000 particle.py:1(<module>)
        1    0.000    0.000    0.000    0.000 particle.py:1(Particle)
     1000    0.000    0.000    0.000    0.000 particle.py:2(__init__)
        1    0.000    0.000    0.000    0.000 particle_simulator.py:1(<module>)
        1    0.000    0.000    0.000    0.000 particle_simulator.py:1(ParticleSimulator)
        1    0.000    0.000    0.000    0.000 particle_simulator.py:2(__init__)
        1    3.572    3.572    3.572    3.572 particle_simulator.py:4(evolve)
        1    0.000    0.000    0.001    0.001 random.py:1(<module>)
        1    0.000    0.000    0.000    0.000 random.py:103(__init_subclass__)
        1    0.000    0.000    0.000    0.000 random.py:123(seed)
     3000    0.001    0.000    0.001    0.000 random.py:415(uniform)
        1    0.000    0.000    0.000    0.000 random.py:709(SystemRandom)
        1    0.000    0.000    0.000    0.000 random.py:78(Random)
        1    0.000    0.000    0.000    0.000 random.py:94(__init__)
        4    0.000    0.000    0.000    0.000 {built-in method _imp._fix_co_filename}
       32    0.000    0.000    0.000    0.000 {built-in method _imp.acquire_lock}
        4    0.000    0.000    0.000    0.000 {built-in method _imp.create_builtin}
        4    0.000    0.000    0.000    0.000 {built-in method _imp.exec_builtin}
        8    0.000    0.000    0.000    0.000 {built-in method _imp.is_builtin}
        4    0.000    0.000    0.000    0.000 {built-in method _imp.is_frozen}
       32    0.000    0.000    0.000    0.000 {built-in method _imp.release_lock}
       16    0.000    0.000    0.000    0.000 {built-in method _thread.allocate_lock}
       16    0.000    0.000    0.000    0.000 {built-in method _thread.get_ident}
        4    0.000    0.000    0.000    0.000 {built-in method builtins.__build_class__}
      5/1    0.000    0.000    3.582    3.582 {built-in method builtins.exec}
       40    0.000    0.000    0.000    0.000 {built-in method builtins.getattr}
       33    0.000    0.000    0.000    0.000 {built-in method builtins.hasattr}
       19    0.000    0.000    0.000    0.000 {built-in method builtins.isinstance}
       16    0.000    0.000    0.000    0.000 {built-in method builtins.len}
        8    0.000    0.000    0.000    0.000 {built-in method builtins.max}
       12    0.000    0.000    0.000    0.000 {built-in method from_bytes}
        4    0.002    0.001    0.002    0.001 {built-in method io.open_code}
        4    0.000    0.000    0.000    0.000 {built-in method marshal.loads}
        1    0.000    0.000    0.000    0.000 {built-in method math.exp}
        2    0.000    0.000    0.000    0.000 {built-in method math.log}
        1    0.000    0.000    0.000    0.000 {built-in method math.sqrt}
       12    0.000    0.000    0.000    0.000 {built-in method posix.fspath}
        4    0.000    0.000    0.000    0.000 {built-in method posix.getcwd}
        1    0.000    0.000    0.000    0.000 {built-in method posix.register_at_fork}
       16    0.003    0.000    0.003    0.000 {built-in method posix.stat}
        1    0.000    0.000    0.000    0.000 {function Random.seed at 0x7f53d66ed790}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
        4    0.000    0.000    0.000    0.000 {method 'endswith' of 'str' objects}
       16    0.000    0.000    0.000    0.000 {method 'get' of 'dict' objects}
       52    0.000    0.000    0.000    0.000 {method 'join' of 'str' objects}
        8    0.000    0.000    0.000    0.000 {method 'pop' of 'dict' objects}
     3000    0.000    0.000    0.000    0.000 {method 'random' of '_random.Random' objects}
        4    0.001    0.000    0.001    0.000 {method 'read' of '_io.BufferedReader' objects}
        8    0.000    0.000    0.000    0.000 {method 'rfind' of 'str' objects}
       32    0.000    0.000    0.000    0.000 {method 'rpartition' of 'str' objects}
       96    0.000    0.000    0.000    0.000 {method 'rstrip' of 'str' objects}
        4    0.000    0.000    0.000    0.000 {method 'startswith' of 'str' objects}
```

> `PROFILLING -s tottime benchmark`

```bash
python3 -m cProfile benchmark.py
```

```text
    --- SALIDA ---

         8070 function calls (8041 primitive calls) in 3.582 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        8    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:103(release)
        8    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:143(__init__)
        8    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:147(__enter__)
        8    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:151(__exit__)
        8    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:157(_get_module_lock)
        8    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:176(cb)
     12/3    0.000    0.000    0.001    0.000 <frozen importlib._bootstrap>:211(_call_with_frames_removed)        
       52    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:222(_verbose_message)
   ...
```

> `PROFILLING -o prof.out benchmark`

```bash
python3 -m cProfile -o prof.out benchmark.py
```

> `PROFILLING programmatic benchmark`

```py
from benchmark import benchmark
import cProfile

pr = cProfile.Profile()
pr.enable()
benchmark()
pr.disable()
pr.print_stats()
```

> `IPython prun`

```py
[1] from benchmark import benchmark
[2] %prun benchmark()
```

## Analizar graficamente los resultados de `cProfile`

> `INSTALL snakeviz`

```bash
python3 -m pip install snakeviz
```

> `VIEW prof.out`

```bash
python3 -m snakeviz prof.out
```


In [ ]:
def generarPrimos(n):
  primos = [2]
  i = 3
  while len(primos) < n:
    esPrimo = True
    for p in primos:
      if i % p == 0:
        esPrimo = False
        break # rompe el for
    if esPrimo:
      primos.append(i)
    i += 1
  return primos

generarPrimos(20)

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71]